# ML Project 4 

`Fake News` isimli veri seti üzerine çalışıyorum. Bu veri setinde, yayımlanan haberlerin sahte mi gerçek mi oldukları 0 ve 1 şeklinde binary olarak etiketlenmiş. Binary Classification modeli kurulması gerekebilir. Ayrıca hiçbir sayısal veri olmadığından, sadece metin verileri olduğundan, NLP kullanmak gerekebilir. 

## Importing Libraries

In [1]:
import os 
os.chdir("C:/Users/ogulc/ÖNEMLİ ÇALIŞMALAR")

In [2]:
import pandas as pd 
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## stopwords (Anlamlı bilgi taşımayan kelimeleri indirme (Bunlar daha sonra çıkarılmak için kullanılacak))

In [3]:
import nltk 
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ogulc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = set(stopwords.words("english"))

In [5]:
print(stop_words)

{'is', 'and', 'to', 'a', 'then', 'isn', 'here', 'ain', 'have', 'ourselves', "mightn't", 'nor', 'been', 'yours', 'does', 'itself', 'between', 'there', "won't", 'now', 'few', 'why', 'ours', 'weren', "shouldn't", 'both', 'whom', "it's", 'any', 'by', "you've", 'him', 'himself', 'mustn', 'under', 'with', 'before', 'was', 'each', 'ma', 'than', 'did', 'through', 'if', 'y', "that'll", 'it', "you'd", "isn't", 'too', 'just', 'had', 'below', 'more', 'again', 'aren', 't', 'those', 'of', 'as', 'up', 'we', 'what', 'shouldn', 're', 'couldn', 'its', 'the', 'same', "she's", "couldn't", 'hasn', "you'll", 'my', 'myself', 'he', 'such', 'won', 'how', 'will', "hasn't", 'having', 'she', 'during', 'were', 'shan', 'doesn', 'against', 'once', 'me', 'be', 'but', 'very', "needn't", 'mightn', 'over', 'her', 's', 'this', 'hers', 'until', 'them', 'o', 'do', "don't", "weren't", 'into', 'who', 'because', 'so', 'an', 'should', 'further', 'no', 'wasn', "wouldn't", 'when', "should've", 'where', 'own', 'themselves', 'll',

+ Bu komut, **nltk (Natural Language Toolkit)** kitaplığını kullanarak durak kelimeler **(stopwords)** listesini indirmenizi sağlar. Durak kelimeler, doğal dil işleme (NLP) görevlerinde genellikle anlamlı bilgi taşımayan ve bu nedenle metin analizi sırasında çıkarılması gereken yaygın olarak kullanılan kelimelerdir.

## Importing Data

In [6]:
project_4_train = pd.read_csv("fakenews_train.csv")
project_4_test = pd.read_csv("fakenews_test.csv")
project_4_submit = pd.read_csv("fakenews_submit.csv")

In [7]:
project_4_train.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [8]:
project_4_test.head(3)

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...


In [9]:
project_4_submit.head(3)

,id,label
0,20800,0
1,20801,1
2,20802,0


## Analysing Train Set

### .info()

In [10]:
project_4_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


### .shape

In [11]:
project_4_train.shape

(20800, 5)

### .columns

In [12]:
project_4_train.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

### .unique() ve nunique()

In [13]:
project_4_train["author"].nunique()

4201

In [14]:
project_4_train["label"].nunique()

2

### .count()

In [15]:
project_4_train.count()

id        20800
title     20242
author    18843
text      20761
label     20800
dtype: int64

### .isnull().sum()

In [16]:
project_4_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [17]:
project_4_train = project_4_train.fillna("")

In [18]:
project_4_train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

### .duplicated() ve drop_duplicated()

In [19]:
project_4_train.duplicated().sum()

np.int64(0)

## Analysing Test Set and Submit

### .info()

In [20]:
project_4_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5078 non-null   object
 2   author  4697 non-null   object
 3   text    5193 non-null   object
dtypes: int64(1), object(3)
memory usage: 162.6+ KB


### .shape

In [21]:
project_4_test.shape

(5200, 4)

### .columns

In [22]:
project_4_test.columns

Index(['id', 'title', 'author', 'text'], dtype='object')

### .unique() ve nunique()

In [23]:
project_4_test["author"].nunique()

1732

### .count()

In [24]:
project_4_test.count()

id        5200
title     5078
author    4697
text      5193
dtype: int64

### .isnull().sum()

In [25]:
project_4_test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [26]:
project_4_test = project_4_test.fillna("")

In [27]:
project_4_test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

### .duplicated() ve drop_duplicated()

In [28]:
project_4_test.duplicated().sum()

np.int64(0)

## Merge 'project_4_test' and 'project_4_submit' 

In [29]:
project_4_submit.head(3)

,id,label
0,20800,0
1,20801,1
2,20802,0


In [30]:
project_4_test["id"] == project_4_submit["id"]

0       True
1       True
2       True
3       True
4       True
        ... 
5195    True
5196    True
5197    True
5198    True
5199    True
Name: id, Length: 5200, dtype: bool

In [31]:
project_4_test_submit = pd.merge(project_4_test, project_4_submit, on = "id")

In [32]:
project_4_test_submit

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",1
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
...,...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...,0
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,0
5198,25998,300 US Marines To Be Deployed To Russian Borde...,,« Previous - Next » 300 US Marines To Be Deplo...,1


## Merging 'project_4_train' and 'project_4_test_submit'

In [33]:
project_4 = pd.concat([project_4_train, project_4_test_submit], ignore_index = True)

In [34]:
project_4

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
25995,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...,0
25996,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1
25997,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,0
25998,25998,300 US Marines To Be Deployed To Russian Borde...,,« Previous - Next » 300 US Marines To Be Deplo...,1


In [35]:
project_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26000 entries, 0 to 25999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      26000 non-null  int64 
 1   title   26000 non-null  object
 2   author  26000 non-null  object
 3   text    26000 non-null  object
 4   label   26000 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1015.8+ KB


In [36]:
project_4.shape

(26000, 5)

In [37]:
project_4.count()

id        26000
title     26000
author    26000
text      26000
label     26000
dtype: int64

In [38]:
project_4.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## Merging 'author name' and 'news title'

In [39]:
project_4["content"] = project_4["author"] + " " + project_4["title"]

In [40]:
project_4.head(3)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...


## Separating the Data

In [41]:
X = project_4.drop("label", axis = 1)

In [42]:
X.head(2)

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."


In [43]:
y = project_4["label"]

In [44]:
y

0        1
1        0
2        1
3        1
4        1
        ..
25995    0
25996    1
25997    0
25998    1
25999    0
Name: label, Length: 26000, dtype: int64

## PorterStemmer (Kelimeleri Köklerine İndirgeme)

+ `PorterStemmer` ile kelimeler köklerine indirgenir.

In [45]:
port_stem = PorterStemmer()

In [46]:
# Stemming fonksiyonu
def stemming(content):
    if isinstance(content, str):
        # Harfler ve boşluk dışındaki karakterleri kaldır
        stemmed_content = re.sub("[^a-zA-Z\s]", " ", content)
        # Metni küçük harfe çevir
        stemmed_content = stemmed_content.lower()
        # Metni kelimelere ayır
        stemmed_content = stemmed_content.split()
        # Durak kelimeleri çıkar ve köklerine indir
        stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stop_words]
        # Kelimeleri tekrar birleştir
        stemmed_content = " ".join(stemmed_content)
        return stemmed_content
    return content

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ogulc\AppData\Local\Temp\ipykernel_3956\2312952140.py:5: SyntaxWarning: invalid escape sequence '\s'
  stemmed_content = re.sub("[^a-zA-Z\s]", " ", content)


In [47]:
project_4["content"] = project_4["content"].apply(stemming)

In [48]:
project_4["content"]

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
25995    jodi rosen bangladeshi traffic jam never end n...
25996    sheryl gay stolberg john kasich sign one abort...
25997    mike mcphate california today exactli sushi ne...
25998                us marin deploy russian border norway
25999        teddi wayn awkward sex onscreen new york time
Name: content, Length: 26000, dtype: object

## Separating Again

In [49]:
X = project_4["content"].values
y = project_4["label"].values

In [50]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'mike mcphate california today exactli sushi new york time',
       'us marin deploy russian border norway',
       'teddi wayn awkward sex onscreen new york time'], dtype=object)

In [51]:
y

array([1, 0, 1, ..., 0, 1, 0])

## TFidfVectorizer (Kelimeleri Ağırlıklandırarak Sayısal bir Vektör Oluşturma)

+ `TFidfVectorizer` ile metinlere ağırlık atanır.

**TF-IDF (Term Frequency-Inverse Document Frequency) Nedir?**

+ TF (Term Frequency): Bir belgedeki bir kelimenin sıklığını ifade eder. Yani, bir kelimenin belge içinde ne kadar sık geçtiğini ölçer.

+ IDF (Inverse Document Frequency): Bir kelimenin belgenin genelinde ne kadar nadir olduğunu ifade eder. Yani, bir kelimenin belgedeki önemini ölçer.

+ TF-IDF: TF ve IDF'nin çarpımı ile elde edilir. Böylece, bir kelimenin bir belgedeki önemini belirlerken, aynı kelimenin genel belge koleksiyonu içindeki yaygınlığını da dikkate alır.

Bu işlem, metin verilerinin sayısal özelliklere dönüştürülmesi ve makine öğrenmesi modelleri tarafından işlenebilmesi için temel bir adımdır. TF-IDF vektörleri, metin sınıflandırma, metin kümeleme, bilgi çıkarma ve diğer doğal dil işleme görevlerinde yaygın olarak kullanılır.

In [52]:
vectorizer = TfidfVectorizer()

In [53]:
vectorizer.fit(X)

TfidfVectorizer()

In [54]:
X = vectorizer.transform(X)

Burada yapılan işlemler sırasıyla şunlar:

+ Adım 1: `vectorizer = TfidfVectorizer()` ile **vectorizer** değişkeni, **TfidfVectorizer()** sınıfının bir örneği olarak oluşturulur. Bu değişken, metinlere ağırlık atanması için kullanılır.
+ Adım 2: `vectorizer.fit(X)` ile daha önce tekrar oluşturulan X değişkenine **TfidfVectorizer** işlevini uygular. Böylece metindeki kelimelerin frekanslarını ve IDF (Inverse Document Frequency) değerlerini hesaplar. Bu adımda, vektörizer öğrenme sürecini tamamlar ve eğitim verilerindeki kelimelerin frekanslarını öğrenir.
+ Adım 3: `X = vectorizer.fit(X)` belirtilen veriyi (X) TF-IDF matrisine dönüştürür. Bu matris, her bir belgedeki (metin örneği) her kelimenin TF-IDF değerlerini içerir. TF-IDF değeri, bir kelimenin bir belgedeki sıklığı ile tüm belgelerdeki o kelimenin sıklığının oranıdır.
Sonuç olarak, X artık TF-IDF matrisi halindedir ve makine öğrenmesi algoritmaları tarafından kullanılmaya hazırdır.

In [55]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 263213 stored elements and shape (26000, 18866)>
  Coords	Values
  (0, 292)	0.2687157895589713
  (0, 2716)	0.3693871882860092
  (0, 3250)	0.2422331119224367
  (0, 3947)	0.36572148274252403
  (0, 4153)	0.26978240443495427
  (0, 5460)	0.23257096486999831
  (0, 7716)	0.21726269299124773
  (0, 8465)	0.24935613493111794
  (0, 9497)	0.290825608206731
  (0, 9798)	0.3693871882860092
  (0, 14828)	0.25354922096224614
  (0, 17288)	0.2781166125472449
  (1, 1635)	0.2955188023757639
  (1, 2063)	0.15482456635576514
  (1, 2427)	0.3785747505554428
  (1, 3085)	0.19140110904389093
  (1, 3913)	0.2637495226080166
  (1, 6041)	0.7148288693257809
  (1, 7506)	0.1909783988504311
  (1, 18506)	0.302866130276713
  (2, 3231)	0.31666291960534937
  (2, 3405)	0.4582385107661265
  (2, 5911)	0.38560410675769785
  (2, 6567)	0.34955443497637173
  (2, 10596)	0.4947014614528309
  :	:
  (25996, 17846)	0.32997491910781823
  (25996, 18716)	0.08342792934674452
  (259

## Splitting Data 

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y ,random_state = 2)

+ `stratify = y` parametresi, veri bölümleme işlemi sırasında sınıf dengesini korumak için kullanılan bir parametredir. Özellikle sınıflandırma problemlerinde önemlidir. Bu parametre, train_test_split fonksiyonuna verilen hedef değişken (y olarak adlandırılan) üzerinden çalışır.

**Stratify Parametresinin İşlevi:**

+ Sınıf Dengesini Koruma:

Eğer stratify = y olarak ayarlanırsa, train_test_split işlemi sırasında, veri kümesinin bölünmesi sırasında her iki bölümde de hedef değişkenin sınıf oranları (yani, her bir sınıfın veri kümesindeki temsil oranı) orijinal veri kümesiyle aynı olacak şekilde korunur.
Örneğin, eğer y hedef değişkeni 0 ve 1 sınıflarından oluşuyorsa ve bu sınıfların oranı %60 ve %40 ise, stratify = y kullanılarak yapılan bölünmede de train ve test veri setlerinde bu oranlar korunur.

+ Önemi:

Sınıf dengesini korumak, modelin eğitim ve test aşamalarında dengeli bir performans sergilemesine yardımcı olur. Eğer sınıflar dengesiz ise (örneğin, bir sınıf diğerinden çok daha fazla temsil ediliyorsa), model bazı sınıfları doğru öğrenemeyebilir veya test ederken yanıltıcı sonuçlar verebilir.

+ Uygulama:

Özellikle nadir sınıfların olduğu dengesiz veri kümelerinde, stratify parametresi kullanılarak her iki veri kümesinde de nadir sınıfların temsil oranının korunması önemlidir. Bu, modelin nadir durumları doğru bir şekilde tanımasına ve sınıflandırmasına yardımcı olabilir.

## Logistic Regression

### Model Training

In [57]:
from sklearn.linear_model import LogisticRegression
lgstc = LogisticRegression()

In [58]:
lgstc.fit(X_train, y_train)

LogisticRegression()

### Model Evaluation

In [59]:
print("Train setindeki doğruluk:", lgstc.score(X_train, y_train))
print("Test setindeki doğruluk:", lgstc.score(X_test, y_test))

Train setindeki doğruluk: 0.9208173076923077
Test setindeki doğruluk: 0.8998076923076923


In [60]:
lgstc_pred = lgstc.predict(X_test)
print("Logistic Regression modelinin Accuracy (Doğruluk) skoru:", accuracy_score(y_test, lgstc_pred))
print("Logistic Regression modelinin Precision (Hassasiyet) skoru:", precision_score(y_test, lgstc_pred))
print("Logistic Regression modelinin Recall (Duyarlılık) skoru:", recall_score(y_test, lgstc_pred))
print("Logistic Regression modelinin F1 skoru:", f1_score(y_test, lgstc_pred))

Logistic Regression modelinin Accuracy (Doğruluk) skoru: 0.8998076923076923
Logistic Regression modelinin Precision (Hassasiyet) skoru: 0.8905563689604685
Logistic Regression modelinin Recall (Duyarlılık) skoru: 0.9163841807909604
Logistic Regression modelinin F1 skoru: 0.9032856877668461


In [61]:
#Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(lgstc, X, y)
print("Her modeldeki doğruluk oranı:", cv_scores)
print("\nModellerin doğruluk oranlarının ortalaması:", cv_scores.mean())

Her modeldeki doğruluk oranı: [0.95807692 0.96115385 0.96269231 0.94557692 0.655     ]

Modellerin doğruluk oranlarının ortalaması: 0.8965
